Identificación de créditos riesgosos usando una red densa en TensorFlow 2.5 --- 30:54 min
===

* Ultima modificación: Mayo 7, 2021 | [YouTube](https://youtu.be/uHRMInxdBrA)

In [1]:
import warnings

warnings.filterwarnings("ignore")

## Descripción del problema

Las entidades financieras desean mejorar sus procedimientos de aprobación de créditos con el fin de disminuir los riesgos de no pago de la deuda, lo que acarrea pérdidas a la entidad. El problema real consiste en poder decidir si se aprueba o no un crédito particular con base en información que puede ser fácilmente recolectada por teléfono o en la web.

Se tiene una muestra de 1000 observaciones. Cada registro contiene 20 atributos que recopilan información tanto sobre el crédito como sobre la salud financiera del solicitante. La información fue recolectada por una firma alemana y se puede descargar de https://archive.ics.uci.edu/ml/datasets/statlog+(german+credit+data).

Los atributos y sus valores son los siguientes:

     Attribute 1:  (qualitative)
     	      Status of existing checking account
     	      A11 :      ... <    0 DM
     	      A12 : 0 <= ... <  200 DM
     	      A13 :      ... >= 200 DM /
     	            salary assignments for at least 1 year
     	      A14 : no checking account

     Attribute 2:  (numerical)
     	      Duration in month

     Attribute 3:  (qualitative)
     	      Credit history
     	      A30 : no credits taken/
     	            all credits paid back duly
     	      A31 : all credits at this bank paid back duly
     	      A32 : existing credits paid back duly till now
     	      A33 : delay in paying off in the past
     	      A34 : critical account/
     	            other credits existing (not at this bank)

     Attribute 4:  (qualitative)
     	      Purpose
     	      A40 : car (new)
     	      A41 : car (used)
     	      A42 : furniture/equipment
     	      A43 : radio/television
     	      A44 : domestic appliances
     	      A45 : repairs
     	      A46 : education
     	      A47 : (vacation - does not exist?)
     	      A48 : retraining
     	      A49 : business
     	      A410 : others

     Attribute 5:  (numerical)
     	      Credit amount

     Attribute 6:  (qualitative)
     	      Savings account/bonds
     	      A61 :          ... <  100 DM
     	      A62 :   100 <= ... <  500 DM
     	      A63 :   500 <= ... < 1000 DM
     	      A64 :          .. >= 1000 DM
     	      A65 :   unknown/ no savings account

     Attribute 7:  (qualitative)
     	      Present employment since
     	      A71 : unemployed
     	      A72 :       ... < 1 year
     	      A73 : 1  <= ... < 4 years  
     	      A74 : 4  <= ... < 7 years
     	      A75 :       .. >= 7 years

     Attribute 8:  (numerical)
     	      Installment rate in percentage of disposable income

     Attribute 9:  (qualitative)
     	      Personal status and sex
     	      A91 : male   : divorced/separated
     	      A92 : female : divorced/separated/married
     	      A93 : male   : single
     	      A94 : male   : married/widowed
     	      A95 : female : single

     Attribute 10: (qualitative)
     	      Other debtors / guarantors
     	      A101 : none
     	      A102 : co-applicant
     	      A103 : guarantor

     Attribute 11: (numerical)
     	      Present residence since

     Attribute 12: (qualitative)
     	      Property
     	      A121 : real estate
     	      A122 : if not A121 : building society savings agreement/
     				   life insurance
     	      A123 : if not A121/A122 : car or other, not in attribute 6
     	      A124 : unknown / no property

     Attribute 13: (numerical)
     	      Age in years

     Attribute 14: (qualitative)
     	      Other installment plans 
     	      A141 : bank
     	      A142 : stores
     	      A143 : none

     Attribute 15: (qualitative)
     	      Housing
     	      A151 : rent
     	      A152 : own
     	      A153 : for free

     Attribute 16: (numerical)
              Number of existing credits at this bank

     Attribute 17: (qualitative)
     	      Job
     	      A171 : unemployed/ unskilled  - non-resident
     	      A172 : unskilled - resident
     	      A173 : skilled employee / official
     	      A174 : management/ self-employed/
     		         highly qualified employee/ officer

     Attribute 18: (numerical)
     	      Number of people being liable to provide maintenance for

     Attribute 19: (qualitative)
     	      Telephone
     	      A191 : none
     	      A192 : yes, registered under the customers name

     Attribute 20: (qualitative)
     	      foreign worker
     	      A201 : yes
     	      A202 : no


## Obtención de datos usando Pandas

In [2]:
import pandas as pd

URL = "https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/german.csv"

df = pd.read_csv(
    URL,
    sep=",",             # separador de campos
    thousands=None,      # separador de miles para números
    decimal=".",         # separador de los decimales para números
    encoding="latin-1",  # idioma
)

#
# Verifica la lectura de los datos
#
df.info()

HTTPError: HTTP Error 404: Not Found

In [ ]:
#
# Contenido del archivo
#
df.head()

## Cantidad de registros por factor

In [ ]:
#
# Variables codificadas como factores
#
for column_type, column_name in zip(df.dtypes, df.dtypes.index):
    if column_type == "object":
        print("-" * 50)
        print(column_name)
        print(df[column_name].value_counts())
        print()

## Rangos de variación de las variables numéricas

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

#
# Los rangos de las variables numéricas son
# bastantes diferentes
#
plt.figure(figsize=(10, 6))
sns.boxplot(data=df)
plt.xticks(rotation=90)

#
# La columna default indica si hubo problemas
# en el pago del préstamo (1- pago, 2- no pago)
# esta es la columna que se desea pronosticar
# 1-si, 2-no
#

## Codificación como factores

In [ ]:
from sklearn.preprocessing import LabelEncoder

#
# Se construye un codificador para transformar
# los strings a enteros (similar a factores en R)
#
enc = LabelEncoder()

for column_type, column_name in zip(df.dtypes, df.dtypes.index):
    if column_type == "object":
        df[column_name] = enc.fit_transform(df[column_name])


#
# Verificación. Las variables que eran categoricas
# ahora son numéricas
#
plt.figure(figsize=(10, 6))
sns.boxplot(data=df)
plt.xticks(rotation=90);

## Escalamiento de las variables

In [ ]:
from sklearn.preprocessing import MinMaxScaler

#
#  Se escalan todas las variables al rango [0, 1]
#  para evitar problemas asociados a la escala
#
scaler = MinMaxScaler()
df_norm = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

plt.figure(figsize=(10, 6))
sns.boxplot(data=df_norm)
plt.xticks(rotation=90);

## Partición de los datos

In [ ]:
#
# Se usa el 90% de los datos para entrenamiento
# y el 10% restante para prueba
#
X_train = df_norm[:900].copy()
X_test = df_norm[900:].copy()

#
# Se elimina la columna default que
# corresponde a la variable de salida
# y se genera la variable dependiente
#
y_train_true = X_train.pop("default")
y_test_true = X_test.pop("default")

## Carga de datos a TensorFlow

In [3]:
import tensorflow as tf

#
# tf.data.Dataset permite escribir data pipelines
#
dataset_train = tf.data.Dataset.from_tensor_slices(
    (X_train.values, y_train_true.values)
)

#
# Por ejemplo, se recorren los primeros
#  cinco patrones
#
for feat, targ in dataset_train.take(5):
    print("Features: {}, Target: {}".format(feat, targ))

2022-05-03 11:25:50.338364: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-03 11:25:50.338416: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


NameError: name 'X_train' is not defined

## Mezcla aletoria de los datos de entrenamiento

In [ ]:
#
# Se mezcla el dataframe y se hace un batch de uno
#
dataset_train = dataset_train.shuffle(len(df_norm)).batch(1)

## Creación y entrenamiento del modelo

In [ ]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(50, activation="relu"),
        tf.keras.layers.Dense(50, activation="relu"),
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)

model.compile(
    optimizer="adam",
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

history = model.fit(
    dataset_train,
    epochs=10,
    verbose=1,
)

## Evaluación

In [ ]:
model.evaluate(
    dataset_train,
    verbose=0,
)

#
#          loss             accuracy
# -----------------------------------

In [ ]:
dataset_test = tf.data.Dataset.from_tensor_slices(
    (X_test.values, y_test_true.values)
).batch(1)

model.evaluate(
    dataset_test,
    verbose=0,
)

#
#          loss             accuracy
# -----------------------------------

## Pronóstico para el conjunto de prueba

In [ ]:
import numpy as np

y_test_pred = model.predict(dataset_test)
y_test_pred = y_test_pred.reshape(-1)
y_test_pred = np.where(y_test_pred < 0.5, 0, 1)
y_test_pred

## Evaluación usando la matriz de confusión

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test_true, y_test_pred)